In [30]:
#take a look of requirements.txt i put some more stuff and also yuo need pip install transformers

In [31]:
!pip install datasets

In [32]:
"""from datasets import load_dataset

dataset = load_dataset('wiki_dpr', trust_remote_code=True)"""


"from datasets import load_dataset\n\ndataset = load_dataset('wiki_dpr', trust_remote_code=True)"

In [33]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import os
from langchain_openai import OpenAIEmbeddings, OpenAI

In [34]:
# Define the path to the folder containing your PDFs
pdf_folder = r"C:\Users\cunia\SPICED_ACADemy\recipe_generator_capstone"

In [35]:
# Find all PDF files in the specified folder
pdf_files = [os.path.join(pdf_folder, file) for file in os.listdir(pdf_folder) if file.endswith(".pdf")]


In [36]:
# Load PDF documents
documents = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(file_path=pdf_file)
    documents.extend(loader.load())


In [37]:
# Split text into segments
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)


In [38]:
# take environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
# Initialize OpenAIEmbeddings with API key
embeddings = OpenAIEmbeddings()

In [40]:
# Create vector store for the documents
vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("faiss_index_recipes")

In [41]:
# Load the FAISS index
new_vectorstore = FAISS.load_local("faiss_index_recipes", embeddings, allow_dangerous_deserialization=True)

In [42]:
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [48]:
# Example query for RAG
query = "what will be the best substitute to egg in north american cuisine based on the common compound?"

In [44]:
combine_docs_chain = create_stuff_documents_chain(
    OpenAI(), retrieval_qa_chat_prompt
)


In [45]:
retrieval_chain = create_retrieval_chain(
    new_vectorstore.as_retriever(), combine_docs_chain
)

In [49]:
res = retrieval_chain.invoke(
    {"input": query}
)

In [50]:
print(res["answer"])


Based on the common compounds present in both egg and other ingredients in the North American Cuisine Report, the best substitute for egg would be milk. Milk has a high percentage of common compounds with egg, making it a suitable replacement in recipes. Other potential substitutes could be butter or wheat.
